In [16]:
# Import pandas
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary
from sklearn.model_selection import train_test_split

In [17]:
# Assign spreadsheet filename to `file`
file = 'Eclipse_4sourcev1.xls'
# Load spreadsheet
xl = pd.ExcelFile(file)
# Load a sheet into a DataFrame by name: df1
df = xl.parse('total')

In [18]:
print('Total: ', len(df))

Total:  7373


In [31]:
class NaiveBayesClassifer:
    def __init__(self):
        self.name = 'Naive Bayes Classifier'
    
    def train (self, X_train, y_train, number_of_unique_words = 100000):
        self.prior = np.zeros(5)
        labels = list(y_train)
        
        for i in range(5):
            self.prior[i] = labels.count(i)/len(labels)
        print(self.prior)
        
        self.mle_summary = np.zeros((number_of_unique_words, 5))
        self.wordCount_summary = np.zeros(5)
        self.uniqueWords_summary = np.zeros(5)
        self.flag_summary = np.zeros((number_of_unique_words, 5))
        
        summaries = list(X_train.loc[:,'summary'])

        self.dictionary = Dictionary()

        for summary in summaries:
            self.dictionary.add_documents([summary.split()])

        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        for key in self.dictionary.keys():
            for i in range(5):
                self.mle_summary[key][labels[i]] = 1
        
        self.uniqueWords_summary
        for i in range(len(summaries)):
            summary = summaries[i]
            for word in summary:
                
                if (self.flag_summary[word][labels[i]] == 0):
                    self.uniqueWords_summary[labels[i]] += 1
                    self.flag_summary[word][labels[i]] = 1
                self.mle_summary[word][labels[i]] += 1 
                self.wordCount_summary[labels[i]] += 1
        
        for i in range(number_of_unique_words):
            for j in range(5):
                self.mle_summary[i][j] += 1
                self.mle_summary[i][j] /= (self.wordCount_summary[j] + self.uniqueWords_summary[j])
        
        self.mle_desc = np.zeros((number_of_unique_words, 5))
        self.wordCount_desc = np.zeros(5)
        
        descriptions = list(X_train.loc[:,'description'])
    
        for description in descriptions:
            self.dictionary.add_documents([description.split()])

        for i in range(len(descriptions)):
            descriptions[i] = self.dictionary.doc2idx(descriptions[i].split())
        
        
        for i in range(len(descriptions)):
            description = descriptions[i]
            for word in description:
                '''
                if (i==0):
                    print(word, self.wordCount_desc[word])
                '''
                self.mle_desc[word][labels[i]] += 1
                self.wordCount_desc[labels[i]] += 1
        
        for i in range(number_of_unique_words):
            for j in range(5):
                self.mle_desc[i][j] += 1
                self.mle_desc[i][j] /= self.wordCount_desc[labels[j]]
        
        pass
    
    def predict (self, X_test):
        y_test = np.ones((X_test.shape[0], 5))
        for i in range(X_test.shape[0]):
            for j in range(5):
                y_test[i][j] = (self.prior[j])
        print(self.prior)
        summaries = list(X_test.loc[:,'summary'])
        for summary in summaries:
            self.dictionary.add_documents([summary.split()])
        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        descriptions = list(X_test.loc[:,'description'])
        for description in descriptions:
            self.dictionary.add_documents([description.split()])
        for i in range(len(descriptions)):
            descriptions[i] = self.dictionary.doc2idx(descriptions[i].split())
        
        for i in range(X_test.shape[0]):
            for j in range(5):
                log_sum_summary = 0
                log_sum_desc = 0
                for word in summaries[i]:
                    if word != -1 and self.mle_summary[word][j] != 0:
                        log_sum_summary += np.log(self.mle_summary[word][j])
                for word in descriptions[i]:
                    if word != -1 and self.mle_desc[word][j] != 0:
                        log_sum_desc += np.log(self.mle_desc[word][j])
                
                y_test[i][j] = np.log(y_test[i][j])
                y_test[i][j] += ((log_sum_summary) )
                
                ''' + (log_sum_desc) '''
        #print(y_test)
        ret_val = np.zeros(len(y_test))
        
        cnt = 0
        for i in range(X_test.shape[0]):
            mx = 0
            mxindex = -1
            for j in range(5):
                if y_test[i][j] != 0:
                    if mxindex == -1 or y_test[i][j] > mx:
                        mxindex = j
                        mx = y_test[i][j]
            if mxindex == -1:
                cnt += 1
                mxindex = cnt % 5
            ret_val[i] = mxindex
        ret_val =  [int(round(x)) for x in ret_val] 
        print('Number of random classifications: ',cnt)
        return list(ret_val)

In [32]:

# create training and testing vars
y = df.loc[:,'label'] - 1
X = df.loc[:, 'summary':'component']

In [33]:
naiveBayesClassifier = NaiveBayesClassifer()
f1_scores_list = []

In [34]:
f1_scores = np.zeros((10, 5))
for i in range(1,11):
    split_point = (len(X)/11) * i
    naiveBayesClassifier.train(X_train = X.loc[0:split_point,], y_train = y.loc[0:split_point, ])
    y_test = naiveBayesClassifier.predict(X_test = X.loc[split_point:, ])
    
    
    y_label = list(y.loc[split_point:, ])
    
    #print(y_test, y_label)
    #print((y_test == y_label).count(True) / len(y_test))
    
    
    true_positive = np.zeros(5)
    
    false_positive = np.zeros(5)
    false_negative = np.zeros(5)
    
    for j in range(len(y_test)):
        if (y_test[j] == y_label[j]):
            true_positive[y_test[j]] += 1
        else:
            false_positive[y_test[j]] += 1
            false_negative[y_label[j]] += 1
    
    precision = np.zeros(5)
    recall = np.zeros(5)
    for j in range(5):
        precision[j] = true_positive[j] / (true_positive[j] + false_positive[j])
        recall[j] = true_positive[j] / (true_positive[j] + false_negative[j])
    
    for j in range(5):
        f1_scores[i-1][j] = (2 * (precision[j] * recall[j])) / (precision[j] + recall[j]) 
    print('Count: Train vs Test')
    print(y_test.count(0), y_label.count(0))
    print(y_test.count(1), y_label.count(1))
    print(y_test.count(2), y_label.count(2))
    print(y_test.count(3), y_label.count(3))
    print(y_test.count(4), y_label.count(4))
    
    print('F-measure after ', i, 'th iteration: ', f1_scores[i-1])
    f1_scores_list.append(f1_scores)
    pass
    

[0.04769001 0.3681073  0.33532042 0.17585693 0.07302534]
[0.04769001 0.3681073  0.33532042 0.17585693 0.07302534]
Number of random classifications:  0
Count: Train vs Test
5850 698
187 1191
337 2756
67 1268
261 789
F-measure after  1 th iteration:  [0.19303604 0.08417997 0.08988038 0.03595506 0.13904762]
[0.05070843 0.31618195 0.35868755 0.18866518 0.0857569 ]
[0.05070843 0.31618195 0.35868755 0.18866518 0.0857569 ]
Number of random classifications:  0
Count: Train vs Test
3903 662
572 1014
600 2500
258 1133
699 723
F-measure after  2 th iteration:  [0.22519168 0.14627995 0.16193548 0.17972682 0.26160338]
[0.0512183  0.27747389 0.37493784 0.18896072 0.10740925]
[0.0512183  0.27747389 0.37493784 0.18896072 0.10740925]
Number of random classifications:  0
Count: Train vs Test
3163 627
709 880
673 2227
290 1006
527 622
F-measure after  3 th iteration:  [0.25013193 0.1535557  0.20689655 0.22685185 0.30983464]
[0.05033557 0.24794929 0.39932886 0.19686801 0.10551827]
[0.05033557 0.24794929 0

In [35]:
np.mean(np.array(f1_scores_list[0]), axis = 0)

array([0.29927012, 0.19800515, 0.22370673, 0.23961686, 0.29768361])

In [ ]:
# v1 = [0.29708908, 0.12242544, 0.12195517, 0.23450966, 0.30316858]
# v2 = [0.2995275 , 0.1467392 , 0.19238292, 0.25886584, 0.30322861]
# v3 = [0.29927012, 0.19800515, 0.22370673, 0.23961686, 0.29768361]